
A continuación se presenta la serie de tiempo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.graphics.tsaplots as plot_acf

In [ ]:
df=pd.read_csv('daily-min-temperatures.csv',sep=';')
df

### Tamaño del conjunto de datos

In [ ]:
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)
df

In [ ]:
df.tail()

In [ ]:
df['Temp'].isnull().values.any()

### Medidas de tendencia central de los valores de la variable **Temp**

In [ ]:
df['Temp'].describe()

In [ ]:
sns.set(rc={'figure.figsize':(16, 8)}, palette='muted', style = "whitegrid")
df.plot(y='Temp')

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Fecha', fontsize = 20)
plt.ylabel('Temp', fontsize = 20)
plt.title('Serie de tiempo', fontsize = 25)
plt.legend(fontsize = 14)
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(16, 8)}, palette='muted', style = "whitegrid")
df.loc['1981-01-01':'1982-12-31', 'Temp'].plot()

In [ ]:
df.loc[:,'Temp'][1000:2000].describe()

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df['Temp'].diff().dropna(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df['Temp'].diff().dropna(), lags=40, ax=ax2)
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df['Temp'], lags=50, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df['Temp'], lags=50, ax=ax2)
plt.show()

In [1]:
modelo=sm.tsa.statespace.SARIMAX(endog=df['Temp'],order=(2,1,2),seasonal_order=(0,0,1,12),trend='c',enforce_invertibility=False)
order=(2,1,0), seasonal_order=(1,1,0,12), simple_differencing=True)
resultados=modelo.fit()
print(resultados.summary())

SyntaxError: unmatched ')' (<ipython-input-1-c71c164d6dc5>, line 2)

### Predicción con redes neuronales
Usando datos históricos desde el año 1981 hasta 1990, se implementará una arquitectura de Redes Neuronales Recurrentes (RNN), usando un tipo especial de éstas conocido como Long Short Term Memory (LSTM).

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint


### Creación de la RNN tipo LSTM

Con la API Keras, se crea la red LSTM con la siguiente configuración:
+**Capa 1:** LSTM con 100 neuronas.
+**Capa 2:** Dropout que regulariza la red durante el entrenamiento, ignorando de manera random algunas neuronas, mejorando el backpropagation.
+**Capa 3:** Dense, se encarga de la multiplicación matricial para dar el valor de salida de la red.

Sólo la capa 1 tiene una función de activación, que por default es la tanh.


3

In [ ]:
# create a small LSTM network
model = Sequential()
model.add(LSTM(100, input_shape=(df.shape[0], df.shape[1]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='random_normal'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
print(model.summary())

In [ ]:
train_size = int(len(df['Temp']) * 0.80)
y_pred=df['Temp'][:train_size]
y_test=df['Temp'][:train_size]

In [ ]:
checkpoint = ModelCheckpoint("weights.best.hdf5", monitor='val_mean_squared_error', verbose=1, save_best_only=True, mode='min')

# Fit
callbacks_list = [checkpoint]
history = model.fit(y_pred, epochs=2500, batch_size=800, verbose=0, callbacks=callbacks_list, validation_split=0.1)